In [ ]:
# Downloading nltk packages
import nltk
nltk.download('all')

In [ ]:
# importing Dataset
import pandas as pd
df=pd.read_csv('./../input/sms-spam-collection-dataset/spam.csv',usecols=['v1','v2'], encoding='latin-1')
df.head()

In [ ]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
corpus=[]

# Get all the sentences from the 2nd column
sentences=df.v2

# I am using stemming for classification
stemmer=PorterStemmer()

# Loop over all the sentences
for i in range(len(sentences)):
  # Remove non - alphabetical characters from the sentence
  sent=re.sub('[^A-Za-z]',' ',sentences[i])
  # Lower the sentence
  sent=sent.lower()
  # Convert the sentence to a list of words
  words=nltk.word_tokenize(sent)
  # Stem each word which is not in stopwords
  words=[stemmer.stem(word) for word in words if word not in stopwords.words('english')]
  # rejoin the sentence
  sent=' '.join(words)
  # add the sentence to the corpus
  corpus.append(sent)
# One hot encoding the target column i.e,1st column
y=pd.get_dummies(df.v1,drop_first=True)

In [ ]:
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
# The dataset is imbalanced
# Random oversampling duplicates examples from the minority class in the training dataset
from imblearn.over_sampling import RandomOverSampler
os=RandomOverSampler(0.93)
X_train_ns,y_train_ns=os.fit_resample(np.array(corpus).reshape((-1,1)),(y.values))

# PLotting the original Spam : Non Spam distribution
print("original Spam : Non Spam distribution 'Imbalanced'",Counter((y.values).reshape((1,-1))[0]))
plt.hist(((y.values).reshape((1,-1))[0]),bins=3)
plt.title("original Spam : Non Spam distribution 'Imbalanced'")
plt.show()

# PLotting the Randomly Oversampled Spam : Non Spam distribution
print("Randomly Oversampled Spam : Non Spam distribution 'Balanced'",Counter((y_train_ns).reshape((1,-1))[0]))
plt.hist((y_train_ns).reshape((1,-1))[0],bins=3)
plt.title("Randomly Oversampled Spam : Non Spam distribution 'Balanced'")
plt.show()

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(ngram_range=(1,2),max_features=5000)
X_train_ns=cv.fit_transform(np.array(X_train_ns).ravel()).toarray()

In [ ]:
# Making a trian test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_train_ns,y_train_ns,test_size=0.2,random_state=42)

# Using Naive bayes Calssifier
from sklearn.naive_bayes import GaussianNB
g=GaussianNB()
g.fit(X_train_ns,y_train_ns)
y_pred=g.predict(X_test)

from sklearn.metrics import accuracy_score,classification_report,f1_score,confusion_matrix
print("Accuracy of the model ==>",accuracy_score(y_test,y_pred))
print("classification_report ==>\n",classification_report(y_test,y_pred))
print("f1_score==>",f1_score(y_test,y_pred))
print("Confusion Matrix ==>\n",confusion_matrix(y_test,y_pred),)
sns.heatmap(confusion_matrix(y_test,y_pred),annot=True)
plt.show()

# Lets gooo 🚀.... We got a model with testing accuracy 99.7%

# Upvote if Like the Work 😋